In [ ]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns 
import re
from ast import literal_eval
warnings.filterwarnings('ignore')
print('pandas:'+pd.__version__)
print('seaborn:'+sns.__version__)  
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer 
import nltk
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
from nltk.corpus import stopwords 
from sklearn.multiclass import OneVsRestClassifier
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from datetime import datetime
import sqlite3
from sqlite3 import Error
import uuid
from time import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

In [39]:
# import pandas as pd
# import numpy as np
# import seaborn as sns 
# from time import time
# import matplotlib.pyplot as plt 
# from sklearn.utils import resample
# import matplotlib.ticker as ticker
# %matplotlib inline 
from sklearn import model_selection
# from sklearn.utils.extmath import density
# from sklearn.metrics import confusion_matrix
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import wordnet
from nltk import word_tokenize
# from nltk.corpus import stopwords
# import warnings
from sklearn.utils import resample
from sklearn.ensemble import ExtraTreesClassifier
# from ast import literal_eval
# import re
# warnings.filterwarnings('ignore')
# print('pandas:'+pd.__version__)
# print('seaborn:'+sns.__version__)  

In [ ]:
# ##Classificadores

# from sklearn.feature_selection import SelectFromModel
# from sklearn.feature_selection import SelectKBest, chi2
# from sklearn.linear_model import RidgeClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.svm import LinearSVC
# from sklearn.linear_model import SGDClassifier
# from sklearn.linear_model import Perceptron
# from sklearn.linear_model import PassiveAggressiveClassifier
# from sklearn.naive_bayes import BernoulliNB, MultinomialNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.neighbors import NearestCentroid
# from sklearn.ensemble import RandomForestClassifier

In [ ]:
results = []
resultsLogisticRegression = []


# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

id_transacao = dt_string#str(uuid.uuid4())
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def save_metrics(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO metrics( classificador, score_f1,
    precision,
    train_time,
    test_time,
    transacao)
    VALUES(?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

def salvar_dado(valor):
    database = r"../data/metricsDB.sqlite"
    print (id)
    # create a database connection
    conn = create_connection(database)
    with conn:
        # create a new project
#         linha = ['SGD Classifier',  0.4503092535879421,  1.9019618034362793,  0.012923479080200195, id]
#         metrica = ('Cool App with SQLite & Python', '2015-01-01', '2015-01-30');
        valor.append(id_transacao)
        print(valor)
        metrica = save_metrics( conn,valor) 
    print (id)
        

In [3]:
tmdb = pd.read_csv('../../../dados/tmdb_5000_movies.csv')

In [4]:
tmdb['genres'] = tmdb['genres'].fillna('[]')\
.apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
tmdb.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [ ]:
tmdb['genres']=tmdb['genres'].apply(lambda x: '|'.join(x))
tmdb.head(3)

In [ ]:
# generos = tmdb.genres.str.get_dummies()
# generos.head()

In [5]:
tmdb.isnull().sum()


budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [6]:
title_df = tmdb[["overview","genres","title", "keywords"]] #, "all_text"
filmes_balanceados = title_df.copy(deep=True)

In [7]:
lst_col = 'genres'

filmes_balanceados2 = pd.DataFrame({
      col:np.repeat(filmes_balanceados[col].values, filmes_balanceados[lst_col].str.len())
      for col in filmes_balanceados.columns.drop(lst_col)}
    ).assign(**{
    lst_col:np.concatenate(filmes_balanceados[lst_col].values)})[filmes_balanceados.columns]

In [8]:
title_df.head(3)

,overview,genres,title,keywords
0,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]",Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]",Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]",Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."


In [9]:
filmes_balanceados2.tail(3)

,overview,genres,title,keywords
12157,"""Signed, Sealed, Delivered"" introduces a dedic...",Romance,"Signed, Sealed, Delivered","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam..."
12158,"""Signed, Sealed, Delivered"" introduces a dedic...",TV Movie,"Signed, Sealed, Delivered","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam..."
12159,Ever since the second grade when he first saw ...,Documentary,My Date with Drew,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 224..."


In [10]:
filmes_balanceados2['genres'].value_counts()

Drama              2297
Comedy             1722
Thriller           1274
Action             1154
Romance             894
Adventure           790
Crime               696
Science Fiction     535
Horror              519
Family              513
Fantasy             424
Mystery             348
Animation           234
History             197
Music               185
War                 144
Documentary         110
Western              82
Foreign              34
TV Movie              8
Name: genres, dtype: int64

In [ ]:
# dados_dos_filmes= tmdb['overview']
# # dados_dos_filmes.drop(['budget','homepage','id','keywords','original_language',
# #                             'original_title','popularity','production_companies','production_countries',
# #                            'release_date','revenue','runtime','spoken_languages','status','tagline',
# #                            'title','vote_average','vote_count', 'overview','genres' ], axis=1)

In [ ]:
# generos = tmdb.genres.str.get_dummies()
# generos.head()

# #CONCATENA OS DOIS DATAFRAMES
# dados_dos_filmes = pd.concat([tmdb, generos], axis=1)
# dados_dos_filmes.head()

In [ ]:
# df_genres =dados_dos_filmes.drop(['budget','homepage','id','keywords','original_language',
#                             'original_title','popularity','production_companies','production_countries',
#                            'release_date','revenue','runtime','spoken_languages','status','tagline',
#                            'title','vote_average','vote_count', 'overview','genres' ], axis=1)
# df_genres

In [ ]:
# df_genres =dados_dos_filmes.drop(['budget','homepage','id','keywords','original_language',
#                             'original_title','popularity','production_companies','production_countries',
#                            'release_date','revenue','runtime','spoken_languages','status','tagline',
#                            'title','vote_average','vote_count', 'overview','genres' ], axis=1)
# counts = []
# categories = list(df_genres.columns)
# for i in categories:
#     counts.append((i, df_genres[i].sum()))
# df_stats = pd.DataFrame(counts, columns=['genre', '#movies'])
# df_stats

In [ ]:
# dados_dos_filmes.drop(['budget','homepage','id','keywords','original_language',
#                             'original_title','popularity','production_companies','production_countries',
# #                            'release_date','revenue','runtime','spoken_languages','status','tagline',
#                            'title','vote_average','vote_count', 'overview','genres' ], axis=1)

In [ ]:

# # split the data, leave 1/3 out for testing
# data_x = dados_dos_filmes[['overview']]
# data_y = dados_dos_filmes.drop(['budget','homepage','id','keywords','original_language',
#                             'original_title','popularity','production_companies','production_countries',
#                            'release_date','revenue','runtime','spoken_languages','status','tagline',
#                            'title','vote_average','vote_count', 'overview','genres' ], axis=1)
# # stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=0.33)
# xtrain, xval, ytrain, yval = model_selection.train_test_split(data_x, data_y, test_size=0.33, random_state=512)
# # transform matrix of plots into lists to pass to a TfidfVectorizer
# train_x = [x[0].strip() for x in xtrain.values.tolist()]
# test_x = [x[0].strip() for x in xval.values.tolist()] 

#### Separando cada gênero em dataset diferente para poder fazer resample dos dados - Tratamento de classes desbalanceadas

In [11]:
df_Drama = filmes_balanceados2[filmes_balanceados2['genres']=='Drama']
df_Comedy  = filmes_balanceados2[filmes_balanceados2['genres']=='Comedy']
df_Thriller   = filmes_balanceados2[filmes_balanceados2['genres']=='Thriller']
df_Action  = filmes_balanceados2[filmes_balanceados2['genres']=='Action']
df_Romance   = filmes_balanceados2[filmes_balanceados2['genres']=='Romance']
df_Adventure = filmes_balanceados2[filmes_balanceados2['genres']=='Adventure']
df_Crime    = filmes_balanceados2[filmes_balanceados2['genres']=='Crime']
df_Science_Fiction = filmes_balanceados2[filmes_balanceados2['genres']=='Science Fiction']
df_Horror     = filmes_balanceados2[filmes_balanceados2['genres']=='Horror']
df_Family           = filmes_balanceados2[filmes_balanceados2['genres']=='Family']
df_Fantasy    = filmes_balanceados2[filmes_balanceados2['genres']=='Fantasy']
df_Mystery      = filmes_balanceados2[filmes_balanceados2['genres']=='Mystery']
df_Animation   = filmes_balanceados2[filmes_balanceados2['genres']=='Animation']
df_History       = filmes_balanceados2[filmes_balanceados2['genres']=='History']
df_Music         = filmes_balanceados2[filmes_balanceados2['genres']=='Music']
df_War        = filmes_balanceados2[filmes_balanceados2['genres']=='War']
df_Documentary  = filmes_balanceados2[filmes_balanceados2['genres']=='Documentary']
df_Western     = filmes_balanceados2[filmes_balanceados2['genres']=='Western']
df_Foreign     = filmes_balanceados2[filmes_balanceados2['genres']=='Foreign']
df_TV_Movie  = filmes_balanceados2[filmes_balanceados2['genres']=='TV Movie']

In [16]:
df_Comedy  = resample(df_Comedy,
                      replace=True,
                      n_samples=2297,
                      random_state=123)
df_Thriller   = resample(df_Thriller,
                         replace=True,
                         n_samples=2297,
                         random_state=123)
df_Action  = resample(df_Action,
                      replace=True,
                      n_samples=2297,
                      random_state=123)
df_Romance   = resample(df_Romance,
                        replace=True,
                        n_samples=2297,
                        random_state=123)
df_Adventure= resample(df_Adventure, 
                       replace=True,  
                       n_samples=2297,    
                       random_state=123)
df_Crime   = resample(df_Crime,
                      replace=True,     
                      n_samples=2297,    
                      random_state=123)
df_Science_Fiction = resample(df_Science_Fiction,
                              replace=True, 
                              n_samples=2297,
                              random_state=123)
df_Horror     = resample(df_Horror, 
                         replace=True,     
                         n_samples=2297,    
                         random_state=123)
df_Family          = resample(df_Family, 
                              replace=True,
                              n_samples=2297,  
                              random_state=123)
df_Fantasy   = resample(df_Fantasy, 
                                 replace=True,     
                                 n_samples=2297,    
                                 random_state=123)
df_Mystery      = resample(df_Mystery,
                           replace=True,     
                           n_samples=2297,   
                           random_state=123)
df_Animation  = resample(df_Animation, 
                         replace=True, 
                         n_samples=2297, 
                         random_state=123)
df_History    = resample(df_History, 
                         replace=True, 
                         n_samples=2297,  
                         random_state=123)
df_Music      = resample(df_Music,
                         replace=True,  
                         n_samples=2297, 
                         random_state=123)
df_War       = resample(df_War, 
                        replace=True,   
                        n_samples=2297,    
                        random_state=123)
df_Documentary= resample(df_Documentary, 
                         replace=True,
                         n_samples=2297,    
                         random_state=123)
df_Western    = resample(df_Western, 
                         replace=True,     
                         n_samples=2297,    
                         random_state=123)
df_Foreign   = resample(df_Foreign, 
                        replace=True,
                        n_samples=2297,  
                        random_state=123)
df_TV_Movie = resample(df_TV_Movie, 
                       replace=True, 
                       n_samples=2297,
                       random_state=123)

In [17]:
df_upsampled = pd.concat([df_Drama, df_Comedy], axis=0 )
df_upsampled = pd.concat([df_upsampled, df_Thriller ]
                         )

df_upsampled = pd.concat([df_upsampled, df_Adventure], axis=0 )
df_upsampled = pd.concat([df_upsampled, df_Action ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Romance ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Crime ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Science_Fiction ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Horror ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Family ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Fantasy ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Mystery ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Animation ]
                         )
df_upsampled = pd.concat([df_upsampled, df_History ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Music ]
                         )
df_upsampled = pd.concat([df_upsampled, df_War ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Documentary ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Western ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Foreign ]
                         )
df_upsampled = pd.concat([df_upsampled, df_TV_Movie ]
                         )

In [18]:
df_upsampled.shape

(45940, 4)

In [19]:
df_upsampled.reset_index()

,index,overview,genres,title,keywords
0,12,Following the death of District Attorney Harve...,Drama,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
1,74,"In 1933 New York, an overly ambitious movie pr...",Drama,King Kong,"[{""id"": 774, ""name"": ""film business""}, {""id"": ..."
2,76,"84 years later, a 101-year-old woman named Ros...",Drama,Titanic,"[{""id"": 2580, ""name"": ""shipwreck""}, {""id"": 298..."
3,139,Life for former United Nations investigator Ge...,Drama,World War Z,"[{""id"": 4565, ""name"": ""dystopia""}, {""id"": 1233..."
4,153,An adaptation of F. Scott Fitzgerald's Long Is...,Drama,The Great Gatsby,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."
...,...,...,...,...,...
45935,9780,"American-born Jayne Valseca, her husband Eduar...",TV Movie,We Have Your Husband,[]
45936,10413,The continued Westward journey of settlers Mis...,TV Movie,Love's Abiding Joy,[]
45937,12158,"""Signed, Sealed, Delivered"" introduces a dedic...",TV Movie,"Signed, Sealed, Delivered","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam..."
45938,8903,A young woman finds out that her mother worked...,TV Movie,Restless,"[{""id"": 470, ""name"": ""spy""}, {""id"": 6091, ""nam..."


In [20]:
df_upsampled['genres'].value_counts()

Animation          2297
Action             2297
Comedy             2297
Horror             2297
Foreign            2297
Romance            2297
War                2297
Fantasy            2297
History            2297
Documentary        2297
Mystery            2297
Family             2297
Crime              2297
Drama              2297
Science Fiction    2297
Adventure          2297
Thriller           2297
Western            2297
TV Movie           2297
Music              2297
Name: genres, dtype: int64

In [24]:
punctuation = """!()-[]{};:'"\, <>./?@#$%^&*_~"""

words = stopwords.words("english")
lemma = nltk.stem.WordNetLemmatizer()
def pre_process(text):
    text = str(text)
    remove_hyperlink = re.sub('http://\S+|https://\S+', '', text)
    for elements in remove_hyperlink:
        if elements in punctuation:
            remove_hyperlink = remove_hyperlink.replace(elements, " ")
    tokens = word_tokenize(remove_hyperlink)
    remove_words = [word for word in tokens if not word in words]
    text = [lemma.lemmatize(word) for word in remove_words]
    joined_words = " ".join(text)
    return joined_words

In [25]:

title_df['overview'] = title_df['overview'].apply(pre_process)
filmes_balanceados['overview'] = filmes_balanceados2['overview'].apply(pre_process)
df_upsampled['overview'] = df_upsampled['overview'].apply(pre_process)


In [26]:
df_upsampled['genres'] = df_upsampled['genres'].map(lambda x: x.split(','))


In [34]:
title_df.head()

,overview,genres,title,keywords
0,In 22nd century paraplegic Marine dispatched m...,"[Action, Adventure, Fantasy, Science Fiction]",Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,Captain Barbossa long believed dead come back ...,"[Adventure, Fantasy, Action]",Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,A cryptic message Bond ’ past sends trail unco...,"[Action, Adventure, Crime]",Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,Following death District Attorney Harvey Dent ...,"[Action, Crime, Drama, Thriller]",The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,John Carter war weary former military captain ...,"[Action, Adventure, Science Fiction]",John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."


In [118]:
# :filmes_balanceados2['genres'] = filmes_balanceados2['genres'].map(lambda x: x.split(''))
title_df['genres'].head() 

0    [Action, Adventure, Fantasy, Science Fiction]
1                     [Adventure, Fantasy, Action]
2                       [Action, Adventure, Crime]
3                 [Action, Crime, Drama, Thriller]
4             [Action, Adventure, Science Fiction]
Name: genres, dtype: object

In [58]:

nltk.download('stopwords') 
words = stopwords.words("english") 
pipeline = Pipeline([
('tfidf', TfidfVectorizer(stop_words=words)),
('clf',  OneVsRestClassifier(ExtraTreesClassifier()) ),
    
])
parameters = { 
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__max_features': [ 5000,10000,15000, 20000],
    "clf__estimator__n_estimators": [10,15,20],
    "clf__estimator__class_weight": ['balanced','balanced_subsample', None],
    'clf__estimator__max_depth':[8,12,15,20],
}

[nltk_data] Downloading package stopwords to /home/igor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Sem tratamento

In [84]:
 
multilabel_binarizer = MultiLabelBinarizer()
# multilabel_binarizer.fit(title_df['genres'])

# transform target variable
y = multilabel_binarizer.fit_transform( title_df['genres']  )

# split the data, leave 1/3 out for testing
data_x = title_df[['overview']]
data_y = y #title_df[[ 'genres' ]]
# stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=0.33)
xtrain, xval, ytrain, yval = model_selection.train_test_split(data_x, data_y, test_size=0.33, random_state=512)
# transform matrix of plots into lists to pass to a TfidfVectorizer
train_x = [x[0].strip() for x in xtrain.values.tolist()]
test_x = [x[0].strip() for x in xval.values.tolist()] 

In [73]:
len(xtrain)

3218

In [74]:
ytrain.shape

(3218, 20)

In [78]:

grid_search_tune = GridSearchCV(
pipeline, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, ytrain )  
print
print("Best parameters set:")
print (grid_search_tune.best_estimator_.steps)

Fitting 2 folds for each of 432 candidates, totalling 864 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:   10.2s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:  1.0min
[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed:  3.2min
[Parallel(n_jobs=2)]: Done 508 tasks      | elapsed:  5.9min
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:  9.5min
[Parallel(n_jobs=2)]: Done 864 out of 864 | elapsed: 10.3min finished


Best parameters set:
[('tfidf', TfidfVectorizer(max_features=15000, ngram_range=(1, 2),
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])), ('clf', OneVsRestClassifier(estimator=ExtraTreesClassifier(class_weight='balanced',
                                                   max_depth=20,
                                                   n_estimators=20)))]


In [79]:
multilabel_binarizer.classes_

array(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western'], dtype=object)

In [80]:

clf  = grid_search_tune.best_estimator_
classes=multilabel_binarizer.classes_
print('_' * 120)
print("Training: ")
print(clf)
t0 = time()  
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro')
print("f1_score:   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        target_names=classes))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))




clf_descr = 'ExtraTreesClassifier-semtratamento-tmdb'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])


________________________________________________________________________________________________________________________
Training: 
Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=15000, ngram_range=(1, 2),
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('clf',
                 OneVsRestClassifier(estimator=ExtraTreesClassifier(class_weight='balanced',
                              

In [81]:
filmes_balanceados2.head(3)

,overview,genres,title,keywords
0,"In the 22nd century, a paraplegic Marine is di...",[Action],Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"In the 22nd century, a paraplegic Marine is di...",[Adventure],Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
2,"In the 22nd century, a paraplegic Marine is di...",[Fantasy],Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."


# Tratamento Parcial

In [125]:
filmes_balanceados2.genres.isna().sum()

0

In [135]:
test_df = filmes_balanceados2[filmes_balanceados2.astype(str)['genres'] != '[]']
test_df

,overview,genres,title,keywords
0,"In the 22nd century, a paraplegic Marine is di...",[Action],Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"In the 22nd century, a paraplegic Marine is di...",[Adventure],Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
2,"In the 22nd century, a paraplegic Marine is di...",[Fantasy],Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
3,"In the 22nd century, a paraplegic Marine is di...",[Science Fiction],Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
4,"Captain Barbossa, long believed to be dead, ha...",[Adventure],Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
...,...,...,...,...
12155,"""Signed, Sealed, Delivered"" introduces a dedic...",[Comedy],"Signed, Sealed, Delivered","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam..."
12156,"""Signed, Sealed, Delivered"" introduces a dedic...",[Drama],"Signed, Sealed, Delivered","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam..."
12157,"""Signed, Sealed, Delivered"" introduces a dedic...",[Romance],"Signed, Sealed, Delivered","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam..."
12158,"""Signed, Sealed, Delivered"" introduces a dedic...",[TV Movie],"Signed, Sealed, Delivered","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam..."


In [133]:
filmes_balanceados2.shape

(12160, 4)

In [131]:
len(list2)

12160

In [136]:
 
multilabel_binarizer = MultiLabelBinarizer()
# multilabel_binarizer.fit(title_df['genres'])

# transform target variable
y = multilabel_binarizer.fit_transform( filmes_balanceados2['genres']  )

# split the data, leave 1/3 out for testing
data_x = filmes_balanceados2[['overview']]
data_y = y #title_df[[ 'genres' ]]
# stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=0.33)
xtrain, xval, ytrain, yval = model_selection.train_test_split(data_x,
                                                              data_y, test_size=0.33, random_state=512)
# transform matrix of plots into lists to pass to a TfidfVectorizer
train_x = [x[0].strip() for x in xtrain.values.astype(str).tolist()]
test_x = [x[0].strip() for x in xval.values.tolist()] 

In [111]:
data_x

,overview
0,"In the 22nd century, a paraplegic Marine is di..."
1,"In the 22nd century, a paraplegic Marine is di..."
2,"In the 22nd century, a paraplegic Marine is di..."
3,"In the 22nd century, a paraplegic Marine is di..."
4,"Captain Barbossa, long believed to be dead, ha..."
...,...
12155,"""Signed, Sealed, Delivered"" introduces a dedic..."
12156,"""Signed, Sealed, Delivered"" introduces a dedic..."
12157,"""Signed, Sealed, Delivered"" introduces a dedic..."
12158,"""Signed, Sealed, Delivered"" introduces a dedic..."


In [106]:
# filmes_balanceados2['genres'] = filmes_balanceados2['genres'].map(lambda x: x.split(','))

# # split the data, leave 1/3 out for testing
# data_x = title_df[['overview']]
# data_y = title_df[[ 'genres' ]]
# # stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=0.33)
# xtrain, xval, ytrain, yval = model_selection.train_test_split(data_x, data_y, test_size=0.33, random_state=512)
# # transform matrix of plots into lists to pass to a TfidfVectorizer
# train_x = [x[0].strip() for x in xtrain.values.tolist()]
# test_x = [x[0].strip() for x in xval.values.tolist()] 

In [107]:
multilabel_binarizer.classes_

array(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western'], dtype=object)

In [137]:

grid_search_tune = GridSearchCV(
pipeline, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, ytrain )  
print
print("Best parameters set:")
print (grid_search_tune.best_estimator_.steps)

Fitting 2 folds for each of 432 candidates, totalling 864 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:   38.5s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:  4.1min
[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed: 14.7min


KeyboardInterrupt: 

In [ ]:

clf  = grid_search_tune.best_estimator_
classes=multilabel_binarizer.classes_
print('_' * 120)
print("Training: ")
print(clf)
t0 = time()  
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro')
print("f1_score:   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        target_names=classes))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))




clf_descr = 'ExtraTreesClassifier-tratamentoParcial-tmdb'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])


# Tratamento UPsampled

In [ ]:
# filmes_balanceados2['genres'] = filmes_balanceados2['genres'].map(lambda x: x.split(','))

multilabel_binarizer = MultiLabelBinarizer()
# multilabel_binarizer.fit(title_df['genres'])

# transform target variable
y = multilabel_binarizer.fit_transform( df_upsampled['genres']  )

# split the data, leave 1/3 out for testing
data_x = df_upsampled[['overview']]
data_y = y #title_df[[ 'genres' ]]
# stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=0.33)
xtrain, xval, ytrain, yval = model_selection.train_test_split(data_x, data_y, test_size=0.33, random_state=512)
# transform matrix of plots into lists to pass to a TfidfVectorizer
train_x = [x[0].strip() for x in xtrain.values.tolist()]
test_x = [x[0].strip() for x in xval.values.tolist()] 

In [ ]:
multilabel_binarizer.classes_

In [ ]:

grid_search_tune = GridSearchCV(
pipeline, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, ytrain )  
print
print("Best parameters set:")
print (grid_search_tune.best_estimator_.steps)

In [ ]:

clf  = grid_search_tune.best_estimator_
classes=multilabel_binarizer.classes_
print('_' * 120)
print("Training: ")
print(clf)
t0 = time()  
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro')
print("f1_score:   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        target_names=classes))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))




clf_descr = 'ExtraTreesClassifier-upsampled-tmdb'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])


# Fazendo nova predição com upsample

In [ ]:
tmdb = pd.read_csv('../../../dados/tmdb_5000_movies.csv')

In [ ]:
tmdb['genres'] = tmdb['genres'].fillna('[]')\
.apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
tmdb.head(3)

In [ ]:
tmdb['overview'].fillna('', inplace=True)
tmdb.isnull().sum()

In [ ]:
overview_df = tmdb[["overview","genres"]]
filmes_balanceados = overview_df.copy(deep=True)

In [ ]:
lst_col = 'genres'

filmes_balanceados2 = pd.DataFrame({
      col:np.repeat(filmes_balanceados[col].values, filmes_balanceados[lst_col].str.len())
      for col in filmes_balanceados.columns.drop(lst_col)}
    ).assign(**{
    lst_col:np.concatenate(filmes_balanceados[lst_col].values)})[filmes_balanceados.columns]

In [ ]:
filmes_balanceados2['genres'].value_counts()

In [ ]:
df_Drama = filmes_balanceados2[filmes_balanceados2['genres']=='Drama']
df_Comedy  = filmes_balanceados2[filmes_balanceados2['genres']=='Comedy']
df_Thriller   = filmes_balanceados2[filmes_balanceados2['genres']=='Thriller']
df_Action  = filmes_balanceados2[filmes_balanceados2['genres']=='Action']
df_Romance   = filmes_balanceados2[filmes_balanceados2['genres']=='Romance']
df_Adventure = filmes_balanceados2[filmes_balanceados2['genres']=='Adventure']
df_Crime    = filmes_balanceados2[filmes_balanceados2['genres']=='Crime']
df_Science_Fiction = filmes_balanceados2[filmes_balanceados2['genres']=='Science Fiction']
df_Horror     = filmes_balanceados2[filmes_balanceados2['genres']=='Horror']
df_Family           = filmes_balanceados2[filmes_balanceados2['genres']=='Family']
df_Fantasy    = filmes_balanceados2[filmes_balanceados2['genres']=='Fantasy']
df_Mystery      = filmes_balanceados2[filmes_balanceados2['genres']=='Mystery']
df_Animation   = filmes_balanceados2[filmes_balanceados2['genres']=='Animation']
df_History       = filmes_balanceados2[filmes_balanceados2['genres']=='History']
df_Music         = filmes_balanceados2[filmes_balanceados2['genres']=='Music']
df_War        = filmes_balanceados2[filmes_balanceados2['genres']=='War']
df_Documentary  = filmes_balanceados2[filmes_balanceados2['genres']=='Documentary']
df_Western     = filmes_balanceados2[filmes_balanceados2['genres']=='Western']
df_Foreign     = filmes_balanceados2[filmes_balanceados2['genres']=='Foreign']
df_TV_Movie  = filmes_balanceados2[filmes_balanceados2['genres']=='TV Movie']

In [ ]:
from sklearn.utils import resample
df_Comedy  = resample(df_Comedy,
                      replace=True,
                      n_samples=2297,
                      random_state=123)
df_Thriller   = resample(df_Thriller,
                         replace=True,
                         n_samples=2297,
                         random_state=123)
df_Action  = resample(df_Action,
                      replace=True,
                      n_samples=2297,
                      random_state=123)
df_Romance   = resample(df_Romance,
                        replace=True,
                        n_samples=2297,
                        random_state=123)
df_Adventure= resample(df_Adventure, 
                       replace=True,  
                       n_samples=2297,    
                       random_state=123)
df_Crime   = resample(df_Crime,
                      replace=True,     
                      n_samples=2297,    
                      random_state=123)
df_Science_Fiction = resample(df_Science_Fiction,
                              replace=True, 
                              n_samples=2297,
                              random_state=123)
df_Horror     = resample(df_Horror, 
                         replace=True,     
                         n_samples=2297,    
                         random_state=123)
df_Family          = resample(df_Family, 
                              replace=True,
                              n_samples=2297,  
                              random_state=123)
df_Fantasy   = resample(df_Fantasy, 
                                 replace=True,     
                                 n_samples=2297,    
                                 random_state=123)
df_Mystery      = resample(df_Mystery,
                           replace=True,     
                           n_samples=2297,   
                           random_state=123)
df_Animation  = resample(df_Animation, 
                         replace=True, 
                         n_samples=2297, 
                         random_state=123)
df_History    = resample(df_History, 
                         replace=True, 
                         n_samples=2297,  
                         random_state=123)
df_Music      = resample(df_Music,
                         replace=True,  
                         n_samples=2297, 
                         random_state=123)
df_War       = resample(df_War, 
                        replace=True,   
                        n_samples=2297,    
                        random_state=123)
df_Documentary= resample(df_Documentary, 
                         replace=True,
                         n_samples=2297,    
                         random_state=123)
df_Western    = resample(df_Western, 
                         replace=True,     
                         n_samples=2297,    
                         random_state=123)
df_Foreign   = resample(df_Foreign, 
                        replace=True,
                        n_samples=2297,  
                        random_state=123)
df_TV_Movie = resample(df_TV_Movie, 
                       replace=True, 
                       n_samples=2297,
                       random_state=123)

In [ ]:
df_upsampled = pd.concat([df_Drama, df_Comedy], axis=0 ) 
df_upsampled = pd.concat([df_upsampled, df_Adventure], axis=0 )


df_upsampled = pd.concat([df_upsampled, df_Thriller ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Action ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Romance ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Crime ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Science_Fiction ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Horror ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Family ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Fantasy ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Mystery ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Animation ]
                         )
df_upsampled = pd.concat([df_upsampled, df_History ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Music ]
                         )
df_upsampled = pd.concat([df_upsampled, df_War ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Documentary ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Western ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Foreign ]
                         )
df_upsampled = pd.concat([df_upsampled, df_TV_Movie ]
                         )

In [ ]:
df_upsampled.reset_index()

In [ ]:
df_upsampled['genres'].value_counts()

In [ ]:
df_upsampled['genres'].unique()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_binarizer3 = MultiLabelBinarizer()
df_upsampled['genres'] = df_upsampled['genres'].map(lambda x: x.split(','))
y =multilabel_binarizer3.fit_transform( df_upsampled['genres'] )
tfidf_vectorizer3 = TfidfVectorizer(max_df=0.7, max_features=25000)
# from sklearn import model_selection
xtrain, xval, ytrain, yval = model_selection.train_test_split(df_upsampled['overview'].values, y, test_size=0.2, random_state=512)
xtrain_tfidf = tfidf_vectorizer3.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer3.transform(xval)
train_x = xtrain_tfidf
test_x = xval_tfidf

In [ ]:
multilabel_binarizer3.classes_

In [ ]:
  
from sklearn.ensemble import ExtraTreesClassifier
nltk.download('stopwords') 
words = stopwords.words("english") 
pipeline = Pipeline([
('tfidf', TfidfVectorizer(stop_words=words)),
('clf',  OneVsRestClassifier(ExtraTreesClassifier()) ),
    
])
parameters = {  
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__max_features': [100, 1000,5000,10000,15000],
    "clf__estimator__n_estimators": [10,15,20],
    "clf__estimator__class_weight": ['balanced','balanced_subsample', None],
    'clf__estimator__max_depth':[8,12,15,20],
} 
# rfc = OneVsRestClassifier(ExtraTreesClassifier())
# parameters = {'n_estimators':[10,15,20],  
#              "class_weight": ['balanced','balanced_subsample', None],
#              'max_depth':[3,5,8]}
    
# clf = GridSearchCV(svc, parameters, cv=2, n_jobs=2, verbose=3) 
 

In [ ]:
multilabel_binarizer3.classes_

In [ ]:
train_x.shape

In [ ]:
ytrain.shape

In [ ]:


classes=multilabel_binarizer3.classes_
print('_' * 120)


t0 = time()  
##############
print("Training: ")
# grid_search_tune = GridSearchCV(
# rfc, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune = GridSearchCV(
pipeline, parameters, cv=2, n_jobs=2, verbose=3)

grid_search_tune.fit(train_x, ytrain )  

clf  = grid_search_tune.best_estimator_
print(clf)
print 
############################
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro') 
print("f1_score:   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 


print('Micro Precision: {:.2f}'.format(precision_score(yval, pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(yval, pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(yval, pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(yval, pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(yval, pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(yval, pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(yval, pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(yval, pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(yval, pred, average='weighted')))

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        target_names=classes))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))


if hasattr(clf, 'coef_'):
    print("dimensionality: %d" % clf.coef_.shape[1])
    print("density: %f" % density(clf.coef_))

clf_descr = 'ExtraTreesClassifier-upsampled-tf-idf-tmdb'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])


In [ ]:
 clf.get_params().keys()

# MovieLens

In [ ]:
movies = pd.read_csv('../../../dados/movielens/filmes_com_overview.csv')

In [ ]:
# movies = movies['genres != (no genres listed)']:
movies.drop(movies[movies.genres == '(no genres listed)'].index, inplace=True)

In [ ]:
movies['genres'] = movies.genres.apply(lambda x: x.split('|'))

In [ ]:
movies.head(3)

In [ ]:
 (movies['genres'].values == '').sum()  

In [ ]:
movies['overview'].fillna('', inplace=True)
movies.isnull().sum()

In [ ]:
movies = movies[movies['genres'] != 'chi Fade to Black Kimi no Na o Yobu) (2008)']
movies = movies[movies['genres'] != ' Battle Force (1978)']
movies = movies[movies['genres'] != 'epe&#351']
movies = movies[movies['genres'] != '(no genres listed)']
movies = movies[movies['genres'] != 'Gate the Movie: The Burden of Déjà vu (2013)']
movies['genres'].value_counts()

In [ ]:
# generos = movies.genres.str.get_dummies()
# generos.head()

In [ ]:
# dados_dos_filmes = pd.concat([movies, generos], axis=1)
# dados_dos_filmes.head()

In [ ]:
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != 'chi Fade to Black Kimi no Na o Yobu) (2008)']
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != ' Battle Force (1978)']
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != 'epe&#351']
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != '(no genres listed)']
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != 'Gate the Movie: The Burden of Déjà vu (2013)']
# dados_dos_filmes['genres'].value_counts()

In [ ]:
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != 'genres']
# dados_dos_filmes = dados_dos_filmes[dados_dos_filmes['genres'] != 'Gate the Movie: The Burden of Déjà vu (2013)']
# dados_dos_filmes.columns
# df_genres.columns

In [ ]:
# dados_dos_filmes.shape

In [ ]:

# df_genres =dados_dos_filmes.drop(['movieId','title', 'overview'  ], axis=1)
# counts = []
# categories = list(df_genres.columns)
# categories = categories[1:]
# for i in categories:
#     counts.append((i, df_genres[i].sum()))
# df_stats = pd.DataFrame(counts, columns=['genre', '#movies'])
# df_stats

In [ ]:
title_df = movies[["overview","genres","title"]]

In [ ]:
filmes_balanceados = title_df.copy(deep=True)

In [ ]:
lst_col = 'genres'

filmes_balanceados2 = pd.DataFrame({
      col:np.repeat(filmes_balanceados[col].values, filmes_balanceados[lst_col].str.len())
      for col in filmes_balanceados.columns.drop(lst_col)}
    ).assign(**{
    lst_col:np.concatenate(filmes_balanceados[lst_col].values)})[filmes_balanceados.columns]

In [ ]:
title_df.head(3)

In [ ]:
filmes_balanceados2.tail(3)

In [ ]:
filmes_balanceados2['genres'].value_counts()

In [ ]:
 filmes_balanceados2 = filmes_balanceados2[filmes_balanceados2['genres'] != 'chi Fade to Black Kimi no Na o Yobu) (2008)']
 filmes_balanceados2 = filmes_balanceados2[filmes_balanceados2['genres'] != ' Battle Force (1978)']
 filmes_balanceados2 = filmes_balanceados2[filmes_balanceados2['genres'] != 'epe&#351']
 filmes_balanceados2 = filmes_balanceados2[filmes_balanceados2['genres'] != '(no genres listed)']
 filmes_balanceados2 = filmes_balanceados2[filmes_balanceados2['genres'] != 'Gate the Movie: The Burden of Déjà vu (2013)']
 filmes_balanceados2['genres'].value_counts()

In [ ]:
df_Drama = filmes_balanceados2[filmes_balanceados2['genres']=='Drama']
df_Comedy  = filmes_balanceados2[filmes_balanceados2['genres']=='Comedy']
df_Thriller   = filmes_balanceados2[filmes_balanceados2['genres']=='Thriller']
df_Romance   = filmes_balanceados2[filmes_balanceados2['genres']=='Romance']
df_Action  = filmes_balanceados2[filmes_balanceados2['genres']=='Action']
df_Horror     = filmes_balanceados2[filmes_balanceados2['genres']=='Horror']
df_Documentary  = filmes_balanceados2[filmes_balanceados2['genres']=='Documentary'] 
df_Crime    = filmes_balanceados2[filmes_balanceados2['genres']=='Crime']
df_Adventure = filmes_balanceados2[filmes_balanceados2['genres']=='Adventure']
df_SciFi = filmes_balanceados2[filmes_balanceados2['genres']=='Sci-Fi']
df_Mystery      = filmes_balanceados2[filmes_balanceados2['genres']=='Mystery']
df_Children    = filmes_balanceados2[filmes_balanceados2['genres']=='Children']
df_Animation   = filmes_balanceados2[filmes_balanceados2['genres']=='Animation']
df_Fantasy    = filmes_balanceados2[filmes_balanceados2['genres']=='Fantasy']
df_War        = filmes_balanceados2[filmes_balanceados2['genres']=='War']
df_Western     = filmes_balanceados2[filmes_balanceados2['genres']=='Western']
df_Musical         = filmes_balanceados2[filmes_balanceados2['genres']=='Musical'] 
df_FilmNoir         = filmes_balanceados2[filmes_balanceados2['genres']=='Film-Noir'] 
df_IMAX    = filmes_balanceados2[filmes_balanceados2['genres']=='IMAX']

In [ ]:
df_Comedy  = resample(df_Comedy,
                      replace=True,
                      n_samples=25112,
                      random_state=123)


df_Thriller   = resample(df_Thriller,
                         replace=True,
                         n_samples=25112,
                         random_state=123)
df_Romance   = resample(df_Romance,
                        replace=True,
                        n_samples=25112,
                        random_state=123)
df_Action  = resample(df_Action,
                      replace=True,
                      n_samples=25112,
                      random_state=123)
df_Horror     = resample(df_Horror, 
                         replace=True,     
                         n_samples=25112,    
                         random_state=123)
df_Documentary= resample(df_Documentary, 
                         replace=True,
                         n_samples=25112,    
                         random_state=123)
df_Crime   = resample(df_Crime,
                      replace=True,     
                      n_samples=25112,    
                      random_state=123)
df_Adventure= resample(df_Adventure, 
                       replace=True,  
                       n_samples=25112,    
                       random_state=123)
df_SciFi = resample(df_SciFi,
                              replace=True, 
                              n_samples=25112,
                              random_state=123)
df_Mystery      = resample(df_Mystery,
                           replace=True,     
                           n_samples=25112,   
                           random_state=123)
df_Children          = resample(df_Children, 
                              replace=True,
                              n_samples=25112,  
                              random_state=123)
df_Animation  = resample(df_Animation, 
                         replace=True, 
                         n_samples=25112, 
                         random_state=123)
df_Fantasy   = resample(df_Fantasy, 
                                 replace=True,     
                                 n_samples=25112,    
                                 random_state=123)
df_War       = resample(df_War, 
                        replace=True,   
                        n_samples=25112,    
                        random_state=123)
df_Western    = resample(df_Western, 
                         replace=True,     
                         n_samples=25112,    
                         random_state=123)
df_Musical      = resample(df_Musical,
                         replace=True,  
                         n_samples=25112, 
                         random_state=123)
df_FilmNoir    = resample(df_FilmNoir, 
                         replace=True, 
                         n_samples=25112,  
                         random_state=123)
df_IMAX   = resample(df_IMAX, 
                        replace=True,
                        n_samples=25112,  
                        random_state=123)


 

In [ ]:
df_upsampled = pd.concat([df_Drama, df_Comedy], axis=0 )
df_upsampled = pd.concat([df_upsampled, df_Thriller ])
df_upsampled = pd.concat([df_upsampled, df_Romance ] )
df_upsampled = pd.concat([df_upsampled, df_Action ]  )
df_upsampled = pd.concat([df_upsampled, df_Horror ] )
df_upsampled = pd.concat([df_upsampled, df_Documentary ]  )
df_upsampled = pd.concat([df_upsampled, df_Crime ] )
df_upsampled = pd.concat([df_upsampled, df_Adventure], axis=0 )
df_upsampled = pd.concat([df_upsampled, df_SciFi ] )
df_upsampled = pd.concat([df_upsampled, df_Mystery ] )
df_upsampled = pd.concat([df_upsampled, df_Children ]  )
df_upsampled = pd.concat([df_upsampled, df_Animation ]  )
df_upsampled = pd.concat([df_upsampled, df_Fantasy ] )
df_upsampled = pd.concat([df_upsampled, df_War ]  )
df_upsampled = pd.concat([df_upsampled, df_Western ] )
df_upsampled = pd.concat([df_upsampled, df_Musical ] )

df_upsampled = pd.concat([df_upsampled, df_FilmNoir ] )
df_upsampled = pd.concat([df_upsampled, df_IMAX ]  ) 

In [ ]:
df_upsampled.reset_index()

In [ ]:
df_upsampled['genres'].value_counts()

In [ ]:
PAROU AQUI

In [ ]:
 # split the data, leave 1/3 out for testing
data_x = dados_dos_filmes[['overview']]
data_y = dados_dos_filmes.drop(['movieId', 'title', 'genres', 'overview'], axis=1)
# stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=0.33)
xtrain, xval, ytrain, yval = model_selection.train_test_split(data_x, data_y, test_size=0.33, random_state=512)
# transform matrix of plots into lists to pass to a TfidfVectorizer
train_x = [x[0].strip() for x in xtrain.values.tolist()]
test_x = [x[0].strip() for x in xval.values.tolist()] 



In [ ]:
data_y.info()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
nltk.download('stopwords') 
words = stopwords.words("english") 
pipeline = Pipeline([
('tfidf', TfidfVectorizer(stop_words=words)),
('rfc', ExtraTreesClassifier() ),
    
])
parameters = {
# 'tfidf__max_df': (0.5, 0.65, 0.75),
# 'tfidf__min_df': (0.15, 0.30, 0.35),
'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
"rfc__n_estimators": [10,15,20],
"rfc__class_weight": ['balanced','balanced_subsample', None],
'rfc__max_depth':[20,25,30],
}

In [ ]:

grid_search_tune = GridSearchCV(
pipeline, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, ytrain )  
print
print("Best parameters set:")
print (grid_search_tune.best_estimator_.steps)

In [ ]:
len(grid_search_tune.best_estimator_.classes_)

In [ ]:

clf  = grid_search_tune.best_estimator_
classes=categories
classes=[ 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
print('_' * 120)
print("Training: ")
print(clf)
t0 = time()  
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro')
print("f1_score:   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        ))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))


if hasattr(clf, 'coef_'):
    print("dimensionality: %d" % clf.coef_.shape[1])
    print("density: %f" % density(clf.coef_))

clf_descr = 'LogisticRegression-upsampled'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])


# Refazer predição com resample

In [ ]:
movies = pd.read_csv('../../../dados/movielens/filmes_com_overview.csv')
# movies = movies['genres != (no genres listed)']:
movies.drop(movies[movies.genres == '(no genres listed)'].index, inplace=True)
movies['genres'].value_counts()

In [ ]:
movies['overview'].fillna('', inplace=True)
movies.isnull().sum()

In [ ]:
movies = movies[movies['genres'] != 'chi Fade to Black Kimi no Na o Yobu) (2008)']
movies = movies[movies['genres'] != ' Battle Force (1978)']
movies = movies[movies['genres'] != 'epe&#351']
movies = movies[movies['genres'] != '(no genres listed)']
movies = movies[movies['genres'] != 'Gate the Movie: The Burden of Déjà vu (2013)']
movies['genres'].value_counts()

In [ ]:
movies.head()

In [ ]:
(movies['genres'].values == '').sum()  


In [ ]:
movies['genres'] = movies['genres'].map(lambda x: x.split('|'))
movies.head()

In [ ]:
overview_df = movies[["overview","genres"]]
filmes_balanceados = overview_df.copy(deep=True)

In [ ]:
lst_col = 'genres'

filmes_balanceados2 = pd.DataFrame({
      col:np.repeat(filmes_balanceados[col].values, filmes_balanceados[lst_col].str.len())
      for col in filmes_balanceados.columns.drop(lst_col)}
    ).assign(**{
    lst_col:np.concatenate(filmes_balanceados[lst_col].values)})[filmes_balanceados.columns]
 

In [ ]:
df_Drama = filmes_balanceados2[filmes_balanceados2['genres']=='Drama']
df_Comedy  = filmes_balanceados2[filmes_balanceados2['genres']=='Comedy']
df_Thriller   = filmes_balanceados2[filmes_balanceados2['genres']=='Thriller']
df_Romance   = filmes_balanceados2[filmes_balanceados2['genres']=='Romance']
df_Action  = filmes_balanceados2[filmes_balanceados2['genres']=='Action']
df_Horror     = filmes_balanceados2[filmes_balanceados2['genres']=='Horror']
df_Documentary  = filmes_balanceados2[filmes_balanceados2['genres']=='Documentary']
df_Crime    = filmes_balanceados2[filmes_balanceados2['genres']=='Crime']
df_Adventure = filmes_balanceados2[filmes_balanceados2['genres']=='Adventure'] 
df_Mystery      = filmes_balanceados2[filmes_balanceados2['genres']=='Mystery'] 
df_Animation   = filmes_balanceados2[filmes_balanceados2['genres']=='Animation']
df_Fantasy    = filmes_balanceados2[filmes_balanceados2['genres']=='Fantasy'] 
df_War        = filmes_balanceados2[filmes_balanceados2['genres']=='War']
df_Western     = filmes_balanceados2[filmes_balanceados2['genres']=='Western']
df_Music         = filmes_balanceados2[filmes_balanceados2['genres']=='Musical']

df_Children           = filmes_balanceados2[filmes_balanceados2['genres']=='Children']
df_SciFi = filmes_balanceados2[filmes_balanceados2['genres']=='Sci-Fi']
df_FilmNoir           = filmes_balanceados2[filmes_balanceados2['genres']=='Film-Noir']
df_IMAX     = filmes_balanceados2[filmes_balanceados2['genres']=='IMAX']


In [ ]:
df_Drama.shape

In [ ]:
df_Comedy  = resample(df_Comedy,
                      replace=True,
                      n_samples=25112,
                      random_state=0)
df_Thriller   = resample(df_Thriller,
                         replace=True,
                         n_samples=25112,
                         random_state=0)
df_Romance   = resample(df_Romance,
                        replace=True,
                        n_samples=25112,
                        random_state=0)
df_Action  = resample(df_Action,
                      replace=True,
                      n_samples=25112,
                      random_state=0)
df_Horror     = resample(df_Horror, 
                         replace=True,     
                         n_samples=25112,    
                         random_state=0)
df_Documentary= resample(df_Documentary, 
                         replace=True,
                         n_samples=25112,    
                         random_state=0)
df_Crime   = resample(df_Crime,
                      replace=True,     
                      n_samples=25112,    
                      random_state=0)
df_Adventure= resample(df_Adventure, 
                       replace=True,  
                       n_samples=25112,    
                       random_state=0)
df_Mystery      = resample(df_Mystery,
                           replace=True,     
                           n_samples=25112,   
                           random_state=0)
df_Animation  = resample(df_Animation, 
                         replace=True, 
                         n_samples=25112, 
                         random_state=0)
df_Fantasy   = resample(df_Fantasy, 
                                 replace=True,     
                                 n_samples=25112,    
                                 random_state=0) 

# df_Family          = resample(df_Family, 
#                               replace=True,
#                               n_samples=25112,  
#                               random_state=0)
df_War       = resample(df_War, 
                        replace=True,   
                        n_samples=25112,    
                        random_state=0)
df_Western    = resample(df_Western, 
                         replace=True,     
                         n_samples=25112,    
                         random_state=0)
df_Music      = resample(df_Music,
                         replace=True,  
                         n_samples=25112, 
                         random_state=0)  
df_Children    = resample(df_Children, 
                         replace=True, 
                         n_samples=25112,  
                         random_state=0) 
df_SciFi   = resample(df_SciFi, 
                        replace=True,
                        n_samples=25112,  
                        random_state=0)
df_FilmNoir   = resample(df_FilmNoir, 
                        replace=True,
                        n_samples=25112,  
                        random_state=0)


df_IMAX = resample(df_IMAX, 
                       replace=True, 
                       n_samples=25112,
                       random_state=0)

In [ ]:
df_upsampled = pd.concat([df_Drama, df_Comedy], axis=0 )
df_upsampled = pd.concat([df_upsampled, df_Thriller ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Action ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Adventure ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Romance ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Crime ]
                         )
df_upsampled = pd.concat([df_upsampled, df_SciFi ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Horror ]
                         ) 
df_upsampled = pd.concat([df_upsampled, df_Fantasy ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Mystery ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Animation ]
                         )  
df_upsampled = pd.concat([df_upsampled, df_Music ]
                         )
df_upsampled = pd.concat([df_upsampled, df_War ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Documentary ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Western ]
                         )  
df_upsampled = pd.concat([df_upsampled, df_Children ]
                         )
df_upsampled = pd.concat([df_upsampled, df_FilmNoir ]
                         )
df_upsampled = pd.concat([df_upsampled, df_IMAX ]
                         )

In [ ]:
df_upsampled.reset_index()

In [ ]:
df_upsampled['genres'].value_counts()

In [ ]:
# split the data, leave 1/3 out for testing
data_x = dados_dos_filmes[['overview']]
data_y = dados_dos_filmes.drop(['movieId', 'title', 'genres', 'overview'], axis=1)
# stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=0.33)
xtrain, xval, ytrain, yval = model_selection.train_test_split(data_x, data_y, test_size=0.33, random_state=512)
# transform matrix of plots into lists to pass to a TfidfVectorizer
train_x = [x[0].strip() for x in xtrain.values.tolist()]
test_x = [x[0].strip() for x in xval.values.tolist()] 



In [ ]:
from sklearn.ensemble import RandomForestClassifier
nltk.download('stopwords') 
words = stopwords.words("english") 
pipeline = Pipeline([
('tfidf', TfidfVectorizer(stop_words=words)),
('rfc', RandomForestClassifier() ),
    
])
parameters = {
# 'tfidf__max_df': (0.5, 0.65, 0.75),
# 'tfidf__min_df': (0.15, 0.30, 0.35),
'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
"rfc__n_estimators": [10,15,20],
"rfc__class_weight": ['balanced','balanced_subsample', None],
'rfc__max_depth':[20,25,30,35,40],
}

In [ ]:

grid_search_tune = GridSearchCV(
pipeline, parameters, cv=2, n_jobs=2, verbose=6)
grid_search_tune.fit(train_x, ytrain )  
print
print("Best parameters set:")
print (grid_search_tune.best_estimator_.steps)

In [ ]:

clf  = grid_search_tune.best_estimator_
# classes=categories
classes=[ 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
print('_' * 120)
print("Training: ")
print(clf)
t0 = time()  
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro')
print("f1_score:   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        target_names=classes))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))


if hasattr(clf, 'coef_'):
    print("dimensionality: %d" % clf.coef_.shape[1])
    print("density: %f" % density(clf.coef_))

clf_descr = 'RandomForest-upsampled-movielens'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])


# Refazer predição com resample e IF-IDF

In [ ]:
df_upsampled = pd.concat([df_Drama, df_Comedy], axis=0 )
df_upsampled = pd.concat([df_upsampled, df_Thriller ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Action ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Adventure ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Romance ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Crime ]
                         )
df_upsampled = pd.concat([df_upsampled, df_SciFi ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Horror ]
                         ) 
df_upsampled = pd.concat([df_upsampled, df_Fantasy ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Mystery ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Animation ]
                         )  
df_upsampled = pd.concat([df_upsampled, df_Music ]
                         )
df_upsampled = pd.concat([df_upsampled, df_War ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Documentary ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Western ]
                         )  
df_upsampled = pd.concat([df_upsampled, df_Children ]
                         )
df_upsampled = pd.concat([df_upsampled, df_FilmNoir ]
                         )
df_upsampled = pd.concat([df_upsampled, df_IMAX ]
                         )

In [ ]:
df_upsampled.reset_index()

In [ ]:
df_upsampled['genres'] = df_upsampled['genres'].map(lambda x: x.split(','))
df_upsampled['genres'].head() 


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_binarizer3 = MultiLabelBinarizer()
y =multilabel_binarizer3.fit_transform(df_upsampled['genres'])
tfidf_vectorizer3 = TfidfVectorizer(max_df=0.7, max_features=5000)
# from sklearn import model_selection
xtrain, xval, ytrain, yval = model_selection.train_test_split(df_upsampled['overview'], y, test_size=0.2, random_state=512)
xtrain_tfidf = tfidf_vectorizer3.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer3.transform(xval)
train_x = xtrain_tfidf
test_x = xval_tfidf

In [ ]:
xtrain_tfidf = tfidf_vectorizer3.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer3.transform(xval)
train_x = xtrain_tfidf
test_x = xval_tfidf
from sklearn.ensemble import RandomForestClassifier
nltk.download('stopwords') 
rfc = RandomForestClassifier()
words = stopwords.words("english") 
parameters = {'n_estimators':[10,15,20],  
             "class_weight": ['balanced','balanced_subsample', None],
             'max_depth':[3,5,8]}
# clf = GridSearchCV(svc, parameters, cv=2, n_jobs=2, verbose=3) 
 

In [ ]:


# classes=multilabel_binarizer3.classes_
classes=[ 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
print('_' * 120)


t0 = time()  
##############
print("Training: ")
grid_search_tune = GridSearchCV(
rfc, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, ytrain )  

clf  = grid_search_tune.best_estimator_
print(clf)
print 
############################
train_time = time() - t0
print("Treino: %0.3fs" % train_time)

t0 = time()
pred = clf.predict(test_x)
test_time = time() - t0
print("testes:  %0.3fs" % test_time)

score = f1_score(yval, pred, average="micro")
precision = precision_score(yval, pred, average='micro') 
print("f1_score :   %0.3f" % score)

print('\nAccuracy: {:.2f}\n'.format(accuracy_score(yval, pred)))
print('\nPrecisao: {:.2f}\n'.format(precision)) 

print('Micro Precision: {:.2f}'.format(precision_score(yval, pred, average='micro')))
print('Macro Precision: {:.2f}'.format(precision_score(yval, pred, average='macro')))
print('Weighted Precision: {:.2f}'.format(precision_score(yval, pred, average='weighted')))


print('Micro F1-score: {:.2f}\n'.format(f1_score(yval, pred, average='micro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(yval, pred, average='macro'))) 
print('Weighted F1-score: {:.2f}'.format(f1_score(yval, pred, average='weighted')))

print('Macro Recall: {:.2f}'.format(recall_score(yval, pred, average='macro'))) 
print('Micro Recall: {:.2f}'.format(recall_score(yval, pred, average='micro')))
print('Weighted Recall: {:.2f}'.format(recall_score(yval, pred, average='weighted')))

print("Relatório de Classificação:")
print(classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                        target_names=classes))

# print("confusion matrix:") 
# print(confusion_matrix(yvalb2.argmax(axis=1), pred.argmax(axis=1)))


if hasattr(clf, 'coef_'):
    print("dimensionality: %d" % clf.coef_.shape[1])
    print("density: %f" % density(clf.coef_))

clf_descr = 'LogisticRegression-upsampled'#str(clf).split('(')[0]

test_time=55555    
results.append([clf_descr, score,precision, train_time, test_time])  
resultsLogisticRegression.append([clf_descr, score,precision, train_time, test_time])  
salvar_dado([clf_descr, score,precision, train_time, test_time])
